Install Boto3

In [ ]:
pip install boto3

Set constants

In [ ]:
import boto3
import json
import getpass

# Set constants
IDENTITY_POOL_ID = '<IDENTITY_POOL_ID>'
AWS_ACCOUNT_ID = '<AWS_ACCOUNT_ID>'
COGNITO_USER_POOL_ID = '<COGNITO_USER_POOL_ID>'
COGNITO_CLIENT_ID = '<COGNITO_CLIENT_ID>'
REGION = 'us-west-2'


Get Cognito ID Token

In [ ]:
# Create Cognito IDP client
client_idp = boto3.client('cognito-idp', region_name=REGION)
 
# Promt the user to enter the username and password
username = input('Enter your Cognito username: ')
password = getpass.getpass('Enter your Cognito password: ')
auth_params = {
    "USERNAME" : username,
    "PASSWORD" : password
}

# Get tokens from Cognito
response = client_idp.initiate_auth(
    AuthFlow = 'USER_PASSWORD_AUTH',
    AuthParameters = auth_params,
    ClientId = COGNITO_CLIENT_ID
)

# Read ID token
id_token = response['AuthenticationResult']['IdToken']
print('ID Token = ' + id_token)

Get temporary AWS credentials

In [ ]:
# Create Cognito Identity client
client_identify = boto3.client('cognito-identity', region_name = REGION)

# Get Identify ID
response_identity_get_id = client_identify.get_id(
            AccountId = AWS_ACCOUNT_ID,
            IdentityPoolId = IDENTITY_POOL_ID,
            Logins = {
                'cognito-idp.us-west-2.amazonaws.com/' + COGNITO_USER_POOL_ID : id_token
            }
) 
IDENTITY_ID = response_identity_get_id['IdentityId']                                                                     
 
    
# Get temporary AWS credentials for identity    
aws_credentials = client_identify.get_credentials_for_identity(
    IdentityId = IDENTITY_ID,
    Logins = {
        'cognito-idp.us-west-2.amazonaws.com/' + COGNITO_USER_POOL_ID : id_token
    }
)

access_key_id = aws_credentials['Credentials']['AccessKeyId']
secret_key = aws_credentials['Credentials']['SecretKey']
session_token = aws_credentials['Credentials']['SessionToken']

print('AWS Access Key ID: ' + access_key_id)
print('AWS Secret Key: ' + secret_key)
print('AWS Session Token: ' + session_token)

Access S3 and Print S3 Bucket Names

In [ ]:
# Create an S3 client
s3_client = boto3.client('s3', 
                      aws_access_key_id = access_key_id,
                      aws_secret_access_key = secret_key,
                      aws_session_token = session_token,
                      region_name = REGION
                      )
    
s3_response = s3_client.list_buckets()

# Print S3 bucket names
print('Existing buckets:')
for bucket in s3_response['Buckets']:
    print(f'  {bucket["Name"]}')